In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWijylaoaknCEjmk1axi09O95Gfreea88xAW86BzFBy5-RbEAUjfUi8
Mounted at /content/drive


In [2]:
import sys
sys.path.insert(0,'/content/drive/My Drive/MSc_Project_Colab/BAD_PyTorch/')

In [3]:
!pip install torchaudio

     |████████████████████████████████| 1.9 MB 6.9 MB/s 


**CNN**

In [4]:
from torch import nn
from torchsummary import summary


class CNNNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding='valid'
            ),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.001),
            nn.MaxPool2d(kernel_size=3)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding='valid'
            ),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.001),
            nn.MaxPool2d(kernel_size=3)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding='valid'
            ),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.001),
            nn.MaxPool2d(kernel_size=(3,1), stride=(3,1), padding=0)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.001),
            nn.MaxPool2d(kernel_size=(3,1), stride=(3,1), padding=0)
        )
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(p=0.5)
        self.linearA = nn.Linear(752, 256)
        self.batchnormA = nn.BatchNorm1d(256)
        self.leakyrelu = nn.LeakyReLU(0.001)
        self.linearB = nn.Linear(256, 32)
        self.batchnormB = nn.BatchNorm1d(32)

        self.linear = nn.Linear(32, 1)
        # self.sigmoid = nn.Sigmoid()

    def forward(self, input_data):
        x = self.conv1(input_data)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.linearA(x)
        x = self.batchnormA(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)
        x = self.linearB(x)
        x = self.batchnormB(x)
        x = self.leakyrelu(x)
        x = self.dropout(x)


        logits = self.linear(x)
        # predictions = self.sigmoid(logits)
        return logits


if __name__ == "__main__":
    cnn = CNNNetwork()
    summary(cnn.cuda(), (1, 64, 431))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 16, 62, 429]             160
       BatchNorm2d-2          [-1, 16, 62, 429]              32
         LeakyReLU-3          [-1, 16, 62, 429]               0
         MaxPool2d-4          [-1, 16, 20, 143]               0
            Conv2d-5          [-1, 16, 18, 141]           2,320
       BatchNorm2d-6          [-1, 16, 18, 141]              32
         LeakyReLU-7          [-1, 16, 18, 141]               0
         MaxPool2d-8            [-1, 16, 6, 47]               0
            Conv2d-9            [-1, 16, 4, 45]           2,320
      BatchNorm2d-10            [-1, 16, 4, 45]              32
        LeakyReLU-11            [-1, 16, 4, 45]               0
        MaxPool2d-12            [-1, 16, 1, 45]               0
           Conv2d-13            [-1, 16, 3, 47]           2,320
      BatchNorm2d-14            [-1, 16

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train the model

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchaudio

from dcasedataset import DCASE_Dataset

ANNOTATIONS_FILE = '/content/drive/My Drive/DCASE_Datasets/labels/BirdVox-DCASE20k.csv'
AUDIO_DIR = '/content/drive/My Drive/DCASE_Datasets/audio/'
SAMPLE_RATE = 22050
DURATION = 10
NUM_SAMPLES = 22050 * DURATION


BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.001

def create_data_loader(train_data, batch_size):
    train_dataloader = DataLoader(train_data, batch_size=batch_size)
    return train_dataloader


def train_single_epoch(model, data_loader, loss_fn, optimiser, device):
    for input, target in data_loader:
        input, target = input.to(device), target.to(device)
        print(target.shape)

        # calculate loss
        prediction = model(input)
        # print(prediction.shape)
        sigmoid = nn.Sigmoid()
        # prediction.unsqueeze(1)
        target = target.unsqueeze_(1)
        target = target.type(torch.cuda.FloatTensor)
        loss = loss_fn(sigmoid(prediction), target)
        print(loss)

        # backpropagate error and update weights
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

    print(f"loss: {loss.item()}")


def train(model, data_loader, loss_fn, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}")
        train_single_epoch(model, data_loader, loss_fn, optimiser, device)
        print("---------------------------")
    print("Finished training")


if __name__ == "__main__":

    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using {device}")

    # instantiate dataset object and create data loader
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )

    dcase = DCASE_Dataset(ANNOTATIONS_FILE,
                            AUDIO_DIR,
                            mel_spectrogram,
                            SAMPLE_RATE,
                            NUM_SAMPLES,
                            device)

    train_dataloader = create_data_loader(dcase, BATCH_SIZE)

    cnn = CNNNetwork().to(device)
    print(cnn)

    # initialise loss funtion + optimiser
    loss_fn = nn.BCELoss()
    optimiser = torch.optim.Adam(cnn.parameters(), 
                                 lr=LEARNING_RATE)

    # train model
    train(cnn, train_dataloader, loss_fn, optimiser, device, EPOCHS)

    # save model
    torch.save(cnn.state_dict(), "/content/drive/My Drive/MSc_Project_Colab/BAD_PyTorch/cnn.pth")
    print("Trained cnn saved at cnn.pth")

Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/DCASE_Datasets/audio/BirdVox-DCASE-20k/c79c0b60-c52f-4c1f-a04e-557ed28e4321.wav
/content/drive/My Drive/DCASE_Datasets/audio/BirdVox-DCASE-20k/c79da3ad-34f1-4eb4-a4b7-ed6acd49266b.wav
/content/drive/My Drive/DCASE_Datasets/audio/BirdVox-DCASE-20k/c7a53c95-3fe9-40c1-a4ec-05fb9dd58176.wav
/content/drive/My Drive/DCASE_Datasets/audio/BirdVox-DCASE-20k/c7a6cd05-fbac-4414-a4d9-425d4cb384bd.wav
/content/drive/My Drive/DCASE_Datasets/audio/BirdVox-DCASE-20k/c7a9ea29-bcd1-4faf-a3be-6950f5cf55cf.wav
/content/drive/My Drive/DCASE_Datasets/audio/BirdVox-DCASE-20k/c7ab35fd-05e5-461b-a7b0-780254974876.wav
/content/drive/My Drive/DCASE_Datasets/audio/BirdVox-DCASE-20k/c7b01cbd-68bd-4167-a762-c870b4074ed7.wav
/content/drive/My Drive/DCASE_Datasets/audio/BirdVox-DCASE-20k/c7b2f0cc-41f0-4350-aec4-ee432c379a49.wav
torch.Size([16])
tensor(0.4910, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
/content/drive/My Drive/DCASE_